In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

test_df = pd.read_csv('test_updated.csv')
train_df = pd.read_csv('train_updated.csv')

target = train_df['RiskFlag']
train_ids = train_df['ProfileID']
test_ids = test_df['ProfileID']

train_features = train_df.drop(columns=['RiskFlag', 'ProfileID'])
test_features = test_df.drop(columns=['ProfileID'])

combined_df = pd.concat([train_features, test_features], axis=0)

# Feature Engineering
combined_df['Loan_to_Income'] = combined_df['RequestedSum'] / combined_df['AnnualEarnings']
combined_df['EMI_Approx'] = combined_df['RequestedSum'] / combined_df['RepayPeriod']
combined_df['EMI_to_Income'] = combined_df['EMI_Approx'] / (combined_df['AnnualEarnings'] / 12)

for col in ['AnnualEarnings', 'RequestedSum']:
    combined_df[col] = np.log1p(combined_df[col])

# Encoding 
education_map = {"High School": 0, "Bachelor's": 1, "Master's": 2, "PhD": 3}
combined_df['QualificationLevel'] = combined_df['QualificationLevel'].map(education_map)

binary_map = {'Yes': 1, 'No': 0}
for col in ['OwnsProperty', 'FamilyObligation', 'JointApplicant']:
    combined_df[col] = combined_df[col].map(binary_map)

combined_df = pd.get_dummies(combined_df, columns=['WorkCategory', 'RelationshipStatus', 'FundUseCase'], drop_first=True)

numeric_cols = combined_df.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
combined_df[numeric_cols] = scaler.fit_transform(combined_df[numeric_cols])

X = combined_df[:len(train_df)]
X_test_submission = combined_df[len(train_df):]
y = target

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, f1_score
from sklearn.utils import resample

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
train_data = pd.concat([X_train, y_train], axis=1)
majority = train_data[train_data.RiskFlag == 0]
minority = train_data[train_data.RiskFlag == 1]

minority_upsampled = resample(minority, replace=True, n_samples=len(majority), random_state=42)
train_upsampled = pd.concat([majority, minority_upsampled])

X_train_up = train_upsampled.drop('RiskFlag', axis=1)
y_train_up = train_upsampled.RiskFlag


mlp = MLPClassifier(hidden_layer_sizes=(128, 64, 32), 
                    activation='relu', 
                    solver='adam', 
                    alpha=0.01,                 
                    batch_size=128,             
                    learning_rate='adaptive',   
                    max_iter=300, 
                    early_stopping=True,
                    random_state=42)

mlp.fit(X_train_up, y_train_up)

y_proba_val = mlp.predict_proba(X_val)[:, 1]

best_threshold = 0.5
best_f1 = 0
for t in np.arange(0.1, 1.0, 0.05):
    preds = (y_proba_val >= t).astype(int)
    score = f1_score(y_val, preds)
    if score > best_f1:
        best_f1 = score
        best_threshold = t

print(f"Best Threshold Found: {best_threshold}")
print(f"Best F1 Score: {best_f1}")

y_pred_optimized = (y_proba_val >= best_threshold).astype(int)
print("Optimized Classification Report")
print(classification_report(y_val, y_pred_optimized))

test_probs = mlp.predict_proba(X_test_submission)[:, 1]
test_predictions = (test_probs >= best_threshold).astype(int)

submission = pd.DataFrame({'ProfileID': test_ids, 'RiskFlag': test_predictions})
submission.to_csv('nn_submission.csv', index=False)

In [ ]:
import joblib
joblib.dump(log_reg, "nn_model_nb.pkl")

In [ ]:
import joblib
from sklearn.metrics import accuracy_score, classification_report

model = joblib.load("nn_model_nb.pkl")   

y_pred = model.predict(X_val)

print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))